In [1]:
import pandas as pd
from basic_preprocess import *

- This notebook is a footprint of basic_prerpcess.py which shows the step or process of preprocess

# aggregate appinfo dataset

- match data types
- preprocess few things
- inner join applist, appinfo, metacritic_products
    - join big pub, esrb rating that manually created

## applist

In [2]:
applist = pd.read_pickle("../data/crawled_data/applist.pickle")

In [3]:
applist

,appid,appname,alltimepick,current
0,730,Counter-Strike: Global Offensive,1308963,350403
1,570,Dota 2,1295114,301215
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,3257248,150919
3,271590,Grand Theft Auto V,364548,124679
4,1172470,Apex Legends,330879,113450
...,...,...,...,...
4814,619730,Mad Driver,1002,0
4815,437390,HoPiKo,1001,0
4816,524680,Escape From Nazi Labs,1000,0
4817,634230,Project First Contact,1000,0


In [4]:
applist.dtypes

appid          object
appname        object
alltimepick    object
current        object
dtype: object

In [5]:
# change data type to int
cols = ["appid","alltimepick","current"]
applist[cols] = applist[cols].astype(int)

## appinfo

In [6]:
appinfo = pd.read_csv("../data/crawled_data/appinfo.csv")
appinfo.columns

Index(['app_name_url', 'appid', 'developer', 'genres', 'publisher',
       'release_date', 'specs', 'sys_requirement', 'title', 'user_tags'],
      dtype='object')

In [7]:
# change col order, select only what I need
cols = ['appid','title','release_date', 'genres', 'developer', 'publisher',
        'specs', 'user_tags', 'sys_requirement']
appinfo = appinfo[cols]
appinfo = appinfo[appinfo["release_date"] != "Soon"] # select only released game
appinfo["release_date"] = pd.to_datetime(appinfo["release_date"], infer_datetime_format=True) # release_date to datetime


In [8]:
appinfo.dtypes

appid                       int64
title                      object
release_date       datetime64[ns]
genres                     object
developer                  object
publisher                  object
specs                      object
user_tags                  object
sys_requirement            object
dtype: object

### appinfo genres,specs, user_tags, sys_requirement column preprocess


#### genres column

In [9]:
appinfo["genres"] = to_list_element(appinfo["genres"])

In [10]:
genres = appinfo["genres"].apply(pd.Series) # expand lists to each column
list_value_count(genres, col_name="genre").head(10)

,genre,0,genre,1,genre,2,genre,3,genre,4,genre,5,genre,6,genre,7,genre,8,genre,9
0,Action,1690,Indie,857.0,Indie,690.0,Strategy,221.0,Strategy,105.0,Strategy,43.0,Strategy,24.0,Early Access,7.0,Strategy,2.0,Early Access,2.0
1,Adventure,487,Adventure,716.0,Strategy,284.0,RPG,209.0,Simulation,89.0,Early Access,40.0,Early Access,12.0,Strategy,5.0,Sports,1.0,NaN,NaN
2,Casual,408,Casual,313.0,RPG,260.0,Indie,154.0,Early Access,80.0,Simulation,24.0,RPG,6.0,Simulation,2.0,Early Access,1.0,NaN,NaN
3,Indie,308,Free to Play,236.0,Simulation,221.0,Simulation,147.0,RPG,77.0,RPG,17.0,Simulation,4.0,Sports,1.0,Web Publishing,1.0,NaN,NaN
4,Strategy,129,Simulation,205.0,Casual,148.0,Early Access,99.0,Massively Multiplayer,25.0,Massively Multiplayer,8.0,Sports,2.0,Movie,1.0,NaN,NaN,NaN,NaN
5,RPG,128,Strategy,201.0,Free to Play,99.0,Massively Multiplayer,82.0,Sports,15.0,Sports,5.0,Game Development,1.0,Utilities,1.0,NaN,NaN,NaN,NaN
6,Simulation,128,RPG,191.0,Massively Multiplayer,92.0,Sports,30.0,Indie,14.0,Web Publishing,5.0,Racing,1.0,RPG,1.0,NaN,NaN,NaN,NaN
7,Free to Play,103,Massively Multiplayer,49.0,Sports,49.0,Free to Play,22.0,Racing,5.0,Racing,4.0,Software Training,1.0,NaN,NaN,NaN,NaN,NaN,NaN
8,Racing,47,Sports,41.0,Early Access,48.0,Racing,16.0,Utilities,4.0,Utilities,2.0,Video Production,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9,Animation & Modeling,24,Racing,25.0,Racing,29.0,Software Training,4.0,Video Production,3.0,Design & Illustration,1.0,Utilities,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Select Action, Indie, Strategy, Early Access, Adventure, Casual, Indie, Simulation, RPG, Free to Play, Racing columns
# make list value into dummy variable 
cols = ["Action", "Indie", "Strategy", "Early Access", "Adventure", "Casual", "Indie", "Simulation", "RPG", "Free to Play", "Racing"]
genre_cols= make_list_to_dummy(appinfo["genres"])[cols]
appinfo = pd.concat([appinfo,genre_cols],axis=1)
appinfo.head()


,appid,title,release_date,genres,developer,publisher,specs,user_tags,sys_requirement,Action,Indie,Strategy,Early Access,Adventure,Casual,Indie,Simulation,RPG,Free to Play,Racing
0,730,Counter-Strike: Global Offensive,2012-08-21,"[Action, Free to Play]",Valve,Valve,"Steam Achievements,Full controller support,Ste...","FPS,Shooter,Multiplayer,Competitive,Action,Tea...","OS: Windows® 7/Vista/XP,Processor: Intel® Core...",1,0,0,0,0,0,0,0,0,1,0
1,570,Dota 2,2013-07-09,"[Action, Free to Play, Strategy]",Valve,Valve,"Steam Trading Cards,Steam Workshop,SteamVR Col...","Free to Play,MOBA,Multiplayer,Strategy,eSports...","OS: Windows 7 or newer,Processor: Dual core fr...",1,0,1,0,0,0,0,0,0,1,0
2,578080,PUBG: BATTLEGROUNDS,2017-12-21,"[Action, Adventure, Massively Multiplayer]","KRAFTON, Inc.","KRAFTON, Inc.","Online PvP,Stats,Remote Play on Phone,Remote P...","Survival,Shooter,Multiplayer,Battle Royale,FPS...",Requires a 64-bit processor and operating syst...,1,0,0,0,1,0,0,0,0,0,0
3,1172470,Apex Legends™,2020-11-04,"[Action, Adventure, Free to Play]",Respawn Entertainment,Electronic Arts,"Online PvP,Online Co-op,Steam Achievements,Ful...","Free to Play,Battle Royale,Multiplayer,Shooter...",A Roster of Legendary Characters -Master a gro...,1,0,0,0,1,0,0,0,0,1,0
4,440,Team Fortress 2,2007-10-10,"[Action, Free to Play]",Valve,Valve,"Cross-Platform Multiplayer,Steam Achievements,...","Free to Play,Hero Shooter,Multiplayer,FPS,Shoo...","OS: Windows® 7 (32/64-bit)/Vista/XP,Processor:...",1,0,0,0,0,0,0,0,0,1,0


### specs & user_tags & sys_requirement

- all this columns follow the same procedure as genres column

#### specs column

In [12]:
pd.set_option('display.max_columns', 500)

In [13]:
appinfo["specs"] = to_list_element(appinfo["specs"])
spesc = appinfo["specs"].apply(pd.Series) # expand lists to each column
list_value_count(spesc, col_name="specs").head()

,specs,0,specs,1,specs,2,specs,3,specs,4,specs,5,specs,6,specs,7,specs,8,specs,9,specs,10,specs,11,specs,12,specs,13,specs,14,specs,15,specs,16,specs,17
0,Single-player,3103.0,Steam Achievements,1582,Steam Trading Cards,767.0,Steam Trading Cards,557.0,Steam Cloud,505.0,Steam Cloud,237.0,Steam Cloud,122.0,Remote Play on TV,80.0,Remote Play Together,59.0,Remote Play Together,34.0,Remote Play Together,34.0,Remote Play Together,19.0,Remote Play Together,20.0,Remote Play Together,13.0,Remote Play Together,4.0,Remote Play Together,3.0,Remote Play Together,1.0,Remote Play Together,1.0
1,Online PvP,142.0,Online PvP,560,Full controller support,594.0,Steam Cloud,404.0,Steam Trading Cards,201.0,Steam Leaderboards,137.0,Remote Play on Tablet,103.0,Remote Play on Tablet,72.0,Remote Play on TV,53.0,Remote Play on TV,34.0,Remote Play on Tablet,20.0,Remote Play on TV,18.0,Remote Play on TV,11.0,Remote Play on TV,5.0,Remote Play on TV,3.0,Remote Play on TV,1.0,Remote Play on TV,1.0,NaN,NaN
2,MMO,135.0,Steam Trading Cards,393,Steam Achievements,304.0,Steam Achievements,260.0,Full controller support,117.0,Steam Trading Cards,111.0,Remote Play on TV,94.0,Remote Play Together,65.0,Remote Play on Tablet,37.0,Remote Play on Tablet,22.0,Remote Play on TV,18.0,Includes level editor,11.0,Includes level editor,7.0,Remote Play on Tablet,5.0,Includes Source SDK,1.0,Remote Play on Tablet,1.0,NaN,NaN,NaN,NaN
3,nan,33.0,Online Co-op,197,Online Co-op,214.0,Partial Controller Support,226.0,Steam Leaderboards,108.0,Remote Play on Phone,87.0,Steam Leaderboards,81.0,Steam Leaderboards,61.0,Steam Cloud,34.0,Includes level editor,18.0,Includes level editor,12.0,Remote Play on Tablet,10.0,Remote Play on Tablet,7.0,Includes level editor,2.0,Remote Play on Phone,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Steam Achievements,27.0,Cross-Platform Multiplayer,86,Partial Controller Support,186.0,Full controller support,151.0,Partial Controller Support,93.0,Remote Play on TV,81.0,Remote Play on Phone,57.0,Steam Cloud,50.0,Steam Leaderboards,23.0,Steam Leaderboards,17.0,Steam Leaderboards,11.0,Steam Leaderboards,6.0,Remote Play on Phone,3.0,Remote Play on Phone,1.0,Remote Play on Tablet,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
cols = ["Single-player", "Online PvP", "MMO", "Cross-Platform Multiplayer","Online Co-op"]
spec_cols= make_list_to_dummy(appinfo["specs"])[cols]
appinfo = pd.concat([appinfo,spec_cols],axis=1)
appinfo.head()

,appid,title,release_date,genres,developer,publisher,specs,user_tags,sys_requirement,Action,Indie,Strategy,Early Access,Adventure,Casual,Indie,Simulation,RPG,Free to Play,Racing,Single-player,Online PvP,MMO,Cross-Platform Multiplayer,Online Co-op
0,730,Counter-Strike: Global Offensive,2012-08-21,"[Action, Free to Play]",Valve,Valve,"[Steam Achievements, Full controller support, ...","FPS,Shooter,Multiplayer,Competitive,Action,Tea...","OS: Windows® 7/Vista/XP,Processor: Intel® Core...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,570,Dota 2,2013-07-09,"[Action, Free to Play, Strategy]",Valve,Valve,"[Steam Trading Cards, Steam Workshop, SteamVR ...","Free to Play,MOBA,Multiplayer,Strategy,eSports...","OS: Windows 7 or newer,Processor: Dual core fr...",1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,578080,PUBG: BATTLEGROUNDS,2017-12-21,"[Action, Adventure, Massively Multiplayer]","KRAFTON, Inc.","KRAFTON, Inc.","[Online PvP, Stats, Remote Play on Phone, Remo...","Survival,Shooter,Multiplayer,Battle Royale,FPS...",Requires a 64-bit processor and operating syst...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,1172470,Apex Legends™,2020-11-04,"[Action, Adventure, Free to Play]",Respawn Entertainment,Electronic Arts,"[Online PvP, Online Co-op, Steam Achievements,...","Free to Play,Battle Royale,Multiplayer,Shooter...",A Roster of Legendary Characters -Master a gro...,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1
4,440,Team Fortress 2,2007-10-10,"[Action, Free to Play]",Valve,Valve,"[Cross-Platform Multiplayer, Steam Achievement...","Free to Play,Hero Shooter,Multiplayer,FPS,Shoo...","OS: Windows® 7 (32/64-bit)/Vista/XP,Processor:...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


#### user_tags
- Not going to use this column because it is pretty much same as genres column

In [15]:
appinfo["user_tags"] = to_list_element(appinfo["user_tags"])
tags = appinfo["user_tags"].apply(pd.Series) # expand lists to each column
list_value_count(tags, col_name="user_tag").head(10)

,user_tag,0,user_tag,1,user_tag,2,user_tag,3,user_tag,4,user_tag,5,user_tag,6,user_tag,7,user_tag,8,user_tag,9,user_tag,10,user_tag,11,user_tag,12,user_tag,13,user_tag,14,user_tag,15,user_tag,16,user_tag,17,user_tag,18,user_tag,19
0,Action,351.0,Indie,385.0,Indie,262.0,Multiplayer,187.0,Multiplayer,145,Singleplayer,186.0,Singleplayer,232.0,Singleplayer,197.0,Singleplayer,208.0,Singleplayer,181.0,Singleplayer,147.0,Singleplayer,144.0,Singleplayer,121.0,Singleplayer,89.0,Singleplayer,76.0,Singleplayer,71.0,Singleplayer,94.0,Singleplayer,73.0,Singleplayer,68.0,Singleplayer,56.0
1,Strategy,311.0,Action,326.0,Action,219.0,Indie,169.0,Singleplayer,140,Multiplayer,143.0,Multiplayer,124.0,Multiplayer,102.0,Multiplayer,90.0,Action,81.0,Action,71.0,Atmospheric,71.0,Atmospheric,69.0,Atmospheric,77.0,Great Soundtrack,57.0,Atmospheric,66.0,Adventure,65.0,Adventure,63.0,Atmospheric,67.0,Multiplayer,53.0
2,Indie,309.0,Adventure,204.0,Adventure,173.0,Action,159.0,Action,131,Indie,110.0,Indie,90.0,Action,88.0,2D,83.0,Adventure,78.0,Co-op,69.0,Co-op,61.0,Adventure,56.0,Co-op,64.0,Story Rich,56.0,Great Soundtrack,62.0,Atmospheric,55.0,Great Soundtrack,62.0,Adventure,59.0,Adventure,47.0
3,Free to Play,277.0,Strategy,196.0,Casual,154.0,Adventure,142.0,Indie,108,Action,97.0,Adventure,80.0,Adventure,84.0,Adventure,79.0,Multiplayer,66.0,Atmospheric,65.0,Action,60.0,Co-op,52.0,Adventure,58.0,Atmospheric,55.0,Adventure,59.0,Co-op,50.0,Atmospheric,55.0,Great Soundtrack,57.0,Great Soundtrack,45.0
4,RPG,222.0,Casual,181.0,Multiplayer,153.0,Singleplayer,103.0,Adventure,106,Adventure,97.0,2D,69.0,Indie,75.0,First-Person,73.0,Co-op,65.0,2D,65.0,2D,56.0,Action,52.0,Great Soundtrack,56.0,Action,45.0,Action,49.0,Great Soundtrack,47.0,Indie,44.0,Indie,45.0,Indie,41.0
5,Casual,194.0,RPG,152.0,Strategy,122.0,RPG,98.0,Simulation,84,Shooter,64.0,Action,66.0,First-Person,70.0,Action,65.0,Indie,55.0,Great Soundtrack,61.0,Great Soundtrack,54.0,Multiplayer,52.0,2D,46.0,Co-op,44.0,Co-op,46.0,Casual,42.0,Strategy,35.0,Action,34.0,Casual,41.0
6,Simulation,160.0,Simulation,132.0,RPG,119.0,Simulation,91.0,RPG,70,2D,56.0,First-Person,59.0,2D,66.0,Indie,62.0,2D,53.0,Adventure,60.0,Indie,53.0,Great Soundtrack,51.0,Action,45.0,Adventure,44.0,Difficult,42.0,Open World,42.0,Multiplayer,31.0,Casual,34.0,Atmospheric,40.0
7,Adventure,152.0,Free to Play,78.0,Simulation,106.0,Strategy,88.0,Strategy,70,Casual,53.0,Co-op,50.0,Story Rich,49.0,Co-op,58.0,First-Person,51.0,Multiplayer,54.0,Adventure,51.0,Indie,50.0,Funny,42.0,Multiplayer,39.0,Funny,37.0,Action,32.0,Sandbox,29.0,Strategy,32.0,Action,39.0
8,Early Access,93.0,Multiplayer,78.0,Open World,67.0,Casual,68.0,Casual,67,First-Person,52.0,Strategy,48.0,Simulation,46.0,Great Soundtrack,44.0,Atmospheric,48.0,First-Person,42.0,Fantasy,45.0,Funny,49.0,Open World,38.0,Difficult,39.0,2D,36.0,Funny,32.0,2D,29.0,Co-op,32.0,Open World,35.0
9,Racing,74.0,FPS,69.0,Puzzle,67.0,Shooter,63.0,Shooter,63,Strategy,51.0,Shooter,46.0,Shooter,43.0,RPG,41.0,Strategy,42.0,Fantasy,41.0,Story Rich,42.0,Story Rich,42.0,Multiplayer,38.0,Funny,36.0,Multiplayer,33.0,Indie,31.0,Difficult,28.0,Controller,31.0,RPG,28.0


In [16]:
# single play means the game is single playable.
# some games provides single play and multi play options together. 
# ex) call of duty have both single play mode and multi play mode.
cols = ["Multiplayer", "Co-op"]
tag_cols= make_list_to_dummy(appinfo["user_tags"])[cols]
appinfo = pd.concat([appinfo,tag_cols],axis=1)
appinfo.head()

,appid,title,release_date,genres,developer,publisher,specs,user_tags,sys_requirement,Action,Indie,Strategy,Early Access,Adventure,Casual,Indie,Simulation,RPG,Free to Play,Racing,Single-player,Online PvP,MMO,Cross-Platform Multiplayer,Online Co-op,Multiplayer,Co-op
0,730,Counter-Strike: Global Offensive,2012-08-21,"[Action, Free to Play]",Valve,Valve,"[Steam Achievements, Full controller support, ...","[FPS, Shooter, Multiplayer, Competitive, Actio...","OS: Windows® 7/Vista/XP,Processor: Intel® Core...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
1,570,Dota 2,2013-07-09,"[Action, Free to Play, Strategy]",Valve,Valve,"[Steam Trading Cards, Steam Workshop, SteamVR ...","[Free to Play, MOBA, Multiplayer, Strategy, eS...","OS: Windows 7 or newer,Processor: Dual core fr...",1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
2,578080,PUBG: BATTLEGROUNDS,2017-12-21,"[Action, Adventure, Massively Multiplayer]","KRAFTON, Inc.","KRAFTON, Inc.","[Online PvP, Stats, Remote Play on Phone, Remo...","[Survival, Shooter, Multiplayer, Battle Royale...",Requires a 64-bit processor and operating syst...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1
3,1172470,Apex Legends™,2020-11-04,"[Action, Adventure, Free to Play]",Respawn Entertainment,Electronic Arts,"[Online PvP, Online Co-op, Steam Achievements,...","[Free to Play, Battle Royale, Multiplayer, Sho...",A Roster of Legendary Characters -Master a gro...,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0
4,440,Team Fortress 2,2007-10-10,"[Action, Free to Play]",Valve,Valve,"[Cross-Platform Multiplayer, Steam Achievement...","[Free to Play, Hero Shooter, Multiplayer, FPS,...","OS: Windows® 7 (32/64-bit)/Vista/XP,Processor:...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1


In [17]:
appinfo[(appinfo["Co-op"] == 1) | (appinfo["Multiplayer"] == 0) | (appinfo["Co-op"] == 0)]

,appid,title,release_date,genres,developer,publisher,specs,user_tags,sys_requirement,Action,Indie,Strategy,Early Access,Adventure,Casual,Indie,Simulation,RPG,Free to Play,Racing,Single-player,Online PvP,MMO,Cross-Platform Multiplayer,Online Co-op,Multiplayer,Co-op
0,730,Counter-Strike: Global Offensive,2012-08-21,"[Action, Free to Play]",Valve,Valve,"[Steam Achievements, Full controller support, ...","[FPS, Shooter, Multiplayer, Competitive, Actio...","OS: Windows® 7/Vista/XP,Processor: Intel® Core...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
1,570,Dota 2,2013-07-09,"[Action, Free to Play, Strategy]",Valve,Valve,"[Steam Trading Cards, Steam Workshop, SteamVR ...","[Free to Play, MOBA, Multiplayer, Strategy, eS...","OS: Windows 7 or newer,Processor: Dual core fr...",1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
2,578080,PUBG: BATTLEGROUNDS,2017-12-21,"[Action, Adventure, Massively Multiplayer]","KRAFTON, Inc.","KRAFTON, Inc.","[Online PvP, Stats, Remote Play on Phone, Remo...","[Survival, Shooter, Multiplayer, Battle Royale...",Requires a 64-bit processor and operating syst...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1
3,1172470,Apex Legends™,2020-11-04,"[Action, Adventure, Free to Play]",Respawn Entertainment,Electronic Arts,"[Online PvP, Online Co-op, Steam Achievements,...","[Free to Play, Battle Royale, Multiplayer, Sho...",A Roster of Legendary Characters -Master a gro...,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0
4,440,Team Fortress 2,2007-10-10,"[Action, Free to Play]",Valve,Valve,"[Cross-Platform Multiplayer, Steam Achievement...","[Free to Play, Hero Shooter, Multiplayer, FPS,...","OS: Windows® 7 (32/64-bit)/Vista/XP,Processor:...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3522,604830,All Alone: VR,2017-03-08,"[Adventure, Indie]",8bit,Bitlock Studio,"[Single-player, Steam Trading Cards]","[Adventure, Indie, VR]","OS: Win 7,Processor: i5,Memory: 4 GB RAM,Graph...",0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
3523,271550,Tribloos 2,2013-01-12,"[Casual, Indie, Strategy]",BumpkinBrothers,BumpkinBrothers,"[Single-player, Steam Achievements, Steam Trad...","[Casual, Indie, Strategy, Time Management]",Beautiful Hand Drawn graphicsOver 90 levels fo...,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0
3524,336520,Might & Magic Heroes Online,2015-11-24,"[Free to Play, Massively Multiplayer, RPG, Str...",Blue Byte,Ubisoft,"[Single-player, MMO, Steam Trading Cards]","[Free to Play, Strategy, Massively Multiplayer...","OS: Windows XP SP3,Processor: 2 GHz Dual Core,...",0,0,1,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0
3525,437390,HoPiKo,2017-01-06,"[Action, Casual, Indie]",Laser Dog,Merge Games,"[Single-player, Steam Achievements, Full contr...","[Indie, Casual, Action, Platformer, Great Soun...",Fast and brutal twitch-based gameplay.Awesome ...,1,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0


In [18]:
appinfo.columns

Index(['appid', 'title', 'release_date', 'genres', 'developer', 'publisher',
       'specs', 'user_tags', 'sys_requirement', 'Action', 'Indie', 'Strategy',
       'Early Access', 'Adventure', 'Casual', 'Indie', 'Simulation', 'RPG',
       'Free to Play', 'Racing', 'Single-player', 'Online PvP', 'MMO',
       'Cross-Platform Multiplayer', 'Online Co-op', 'Multiplayer', 'Co-op'],
      dtype='object')

In [19]:
condition = (appinfo["Online PvP"]==1) | (appinfo["MMO"] == 1) | (appinfo["Cross-Platform Multiplayer"] == 1)| (appinfo["Online Co-op"] == 1)| (appinfo["Multiplayer"] == 1) | (appinfo["Co-op"] == 1)

appinfo["playwithother"] = np.where(condition, 1,0)

In [20]:
appinfo.head()

,appid,title,release_date,genres,developer,publisher,specs,user_tags,sys_requirement,Action,Indie,Strategy,Early Access,Adventure,Casual,Indie,Simulation,RPG,Free to Play,Racing,Single-player,Online PvP,MMO,Cross-Platform Multiplayer,Online Co-op,Multiplayer,Co-op,playwithother
0,730,Counter-Strike: Global Offensive,2012-08-21,"[Action, Free to Play]",Valve,Valve,"[Steam Achievements, Full controller support, ...","[FPS, Shooter, Multiplayer, Competitive, Actio...","OS: Windows® 7/Vista/XP,Processor: Intel® Core...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1
1,570,Dota 2,2013-07-09,"[Action, Free to Play, Strategy]",Valve,Valve,"[Steam Trading Cards, Steam Workshop, SteamVR ...","[Free to Play, MOBA, Multiplayer, Strategy, eS...","OS: Windows 7 or newer,Processor: Dual core fr...",1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1
2,578080,PUBG: BATTLEGROUNDS,2017-12-21,"[Action, Adventure, Massively Multiplayer]","KRAFTON, Inc.","KRAFTON, Inc.","[Online PvP, Stats, Remote Play on Phone, Remo...","[Survival, Shooter, Multiplayer, Battle Royale...",Requires a 64-bit processor and operating syst...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,1
3,1172470,Apex Legends™,2020-11-04,"[Action, Adventure, Free to Play]",Respawn Entertainment,Electronic Arts,"[Online PvP, Online Co-op, Steam Achievements,...","[Free to Play, Battle Royale, Multiplayer, Sho...",A Roster of Legendary Characters -Master a gro...,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0,1
4,440,Team Fortress 2,2007-10-10,"[Action, Free to Play]",Valve,Valve,"[Cross-Platform Multiplayer, Steam Achievement...","[Free to Play, Hero Shooter, Multiplayer, FPS,...","OS: Windows® 7 (32/64-bit)/Vista/XP,Processor:...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,1


#### sys_requirement
- sys_requirement is quite embigous to make dummuy variable
- There are not lots of games that require expansive graphic cards

In [21]:
appinfo["sys_requirement"][0]

'OS: Windows® 7/Vista/XP,Processor: Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 8750 processor or better,Memory: 2 GB RAM,Graphics: Video card must be 256 MB or more and should be a DirectX 9-compatible with support for Pixel Shader 3.0,DirectX: Version 9.0c,Storage: 15 GB available space, OS: MacOS X 10.11 (El Capitan) or later,Processor: Intel Core Duo Processor (2GHz or better),Memory: 2 GB RAM,Graphics: ATI Radeon HD 2400 or better / NVidia 8600M or better,Storage: 15 GB available space, OS: Ubuntu 12.04,Processor: 64-bit Dual core from Intel or AMD at 2.8 GHz,Memory: 4 GB RAM,Graphics: nVidia GeForce 8600/9600GT, ATI/AMD Radeon HD2600/3600 (Graphic Drivers: nVidia 310, AMD 12.11), OpenGL 2.1,Storage: 15 GB available space,Sound Card: OpenAL Compatible Sound Card'

In [22]:
appinfo["sys_requirement"] = to_list_element(appinfo["sys_requirement"])
sys = appinfo["sys_requirement"].apply(pd.Series) # expand lists to each column
sys.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169
0,OS: Windows® 7/Vista/XP,Processor: Intel® Core™ 2 Duo E6600 or AMD Phe...,Memory: 2 GB RAM,Graphics: Video card must be 256 MB or more an...,DirectX: Version 9.0c,Storage: 15 GB available space,OS: MacOS X 10.11 (El Capitan) or later,Processor: Intel Core Duo Processor (2GHz or b...,Memory: 2 GB RAM,Graphics: ATI Radeon HD 2400 or better / NVidi...,Storage: 15 GB available space,OS: Ubuntu 12.04,Processor: 64-bit Dual core from Intel or AMD ...,Memory: 4 GB RAM,Graphics: nVidia GeForce 8600/9600GT,ATI/AMD Radeon HD2600/3600 (Graphic Drivers: ...,AMD 12.11),OpenGL 2.1,Storage: 15 GB available space,Sound Card: OpenAL Compatible Sound Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OS: Windows 7 or newer,Processor: Dual core from Intel or AMD at 2.8 GHz,Memory: 4 GB RAM,Graphics: nVidia GeForce 8600/9600GT,ATI/AMD Radeon HD2600/3600,DirectX: Version 9.0c,Network: Broadband Internet connection,Storage: 15 GB available space,Sound Card: DirectX Compatible,OS: OS X Mavericks 10.9 or newer,Processor: Dual core from Intel,Memory: 4 GB RAM,Graphics: nVidia 320M or higher,or Radeon HD 2400 or higher,or Intel HD 3000 or higher,Network: Broadband Internet connection,Storage: 15 GB available space,OS: Ubuntu 12.04 or newer,Processor: Dual core from Intel or AMD at 2.8 GHz,Memory: 4 GB RAM,Graphics: nVidia Geforce 8600/9600GT (Driver v...,AMD HD 2xxx-4xxx (Driver mesa 10.5.9),AMD HD 5xxx+ (Driver mesa 10.5.9 or Catalyst ...,Intel HD 3000 (Driver mesa 10.6),Network: Broadband Internet connection,Storage: 15 GB available space,Sound Card: OpenAL Compatible Sound Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Requires a 64-bit processor and operating system,OS: 64-bit Windows 7,Windows 8.1,Windows 10,Processor: Intel Core i5-4430 / AMD FX-6300,Memory: 8 GB RAM,Graphics: NVIDIA GeForce GTX 960 2GB / AMD Rad...,DirectX: Version 11,Network: Broadband Internet connection,Storage: 40 GB available space,Requires a 64-bit processor and operating system,OS: 64-bit Windows 7,Windows 8.1,Windows 10,Processor: Intel Core i5-6600K / AMD Ryzen 5 1600,Memory: 16 GB RAM,Graphics: NVIDIA GeForce GTX 1060 3GB / AMD Ra...,DirectX: Version 11,Network: Broadband Internet connection,Storage: 50 GB available space,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [23]:
appinfo["sys_requirement"] = to_list_element(appinfo["sys_requirement"])
sys = appinfo["sys_requirement"].apply(pd.Series) # expand lists to each column
sys = list_value_count(sys, col_name="sys_requirement")

In [24]:
pd.set_option('display.max_columns', 350)
sys[:15].head(20)

,sys_requirement,0,sys_requirement,1,sys_requirement,2,sys_requirement,3,sys_requirement,4,sys_requirement,5,sys_requirement,6,sys_requirement,7,sys_requirement,8,sys_requirement,9,sys_requirement,10,sys_requirement,11,sys_requirement,12,sys_requirement,13,sys_requirement,14,sys_requirement,15,sys_requirement,16,sys_requirement,17,sys_requirement,18,sys_requirement,19,sys_requirement,20,sys_requirement,21,sys_requirement,22,sys_requirement,23,sys_requirement,24,sys_requirement,25,sys_requirement,26,sys_requirement,27,sys_requirement,28,sys_requirement,29,sys_requirement,30,sys_requirement,31,sys_requirement,32,sys_requirement,33,sys_requirement,34,sys_requirement,35,sys_requirement,36,sys_requirement,37,sys_requirement,38,sys_requirement,39,sys_requirement,40,sys_requirement,41,sys_requirement,42,sys_requirement,43,sys_requirement,44,sys_requirement,45,sys_requirement,46,sys_requirement,47,sys_requirement,48,sys_requirement,49,sys_requirement,50,sys_requirement,51,sys_requirement,52,sys_requirement,53,sys_requirement,54,sys_requirement,55,sys_requirement,56,sys_requirement,57,sys_requirement,58,sys_requirement,59,sys_requirement,60,sys_requirement,61,sys_requirement,62,sys_requirement,63,sys_requirement,64,sys_requirement,65,sys_requirement,66,sys_requirement,67,sys_requirement,68,sys_requirement,69,sys_requirement,70,sys_requirement,71,sys_requirement,72,sys_requirement,73,sys_requirement,74,sys_requirement,75,sys_requirement,76,sys_requirement,77,sys_requirement,78,sys_requirement,79,sys_requirement,80,sys_requirement,81,sys_requirement,82,sys_requirement,83,sys_requirement,84,sys_requirement,85,sys_requirement,86,sys_requirement,87,sys_requirement,88,sys_requirement,89,sys_requirement,90,sys_requirement,91,sys_requirement,92,sys_requirement,93,sys_requirement,94,sys_requirement,95,sys_requirement,96,sys_requirement,97,sys_requirement,98,sys_requirement,99,sys_requirement,100,sys_requirement,101,sys_requirement,102,sys_requirement,103,sys_requirement,104,sys_requirement,105,sys_requirement,106,sys_requirement,107,sys_requirement,108,sys_requirement,109,sys_requirement,110,sys_requirement,111,sys_requirement,112,sys_requirement,113,sys_requirement,114,sys_requirement,115,sys_requirement,116,sys_requirement,117,sys_requirement,118,sys_requirement,119,sys_requirement,120,sys_requirement,121,sys_requirement,122,sys_requirement,123,sys_requirement,124,sys_requirement,125,sys_requirement,126,sys_requirement,127,sys_requirement,128,sys_requirement,129,sys_requirement,130,sys_requirement,131,sys_requirement,132,sys_requirement,133,sys_requirement,134,sys_requirement,135,sys_requirement,136,sys_requirement,137,sys_requirement,138,sys_requirement,139,sys_requirement,140,sys_requirement,141,sys_requirement,142,sys_requirement,143,sys_requirement,144,sys_requirement,145,sys_requirement,146,sys_requirement,147,sys_requirement,148,sys_requirement,149,sys_requirement,150,sys_requirement,151,sys_requirement,152,sys_requirement,153,sys_requirement,154,sys_requirement,155,sys_requirement,156,sys_requirement,157,sys_requirement,158,sys_requirement,159,sys_requirement,160,sys_requirement,161,sys_requirement,162,sys_requirement,163,sys_requirement,164,sys_requirement,165,sys_requirement,166,sys_requirement,167,sys_requirement,168,sys_requirement,169
0,['Requires a 64-bit processor and operating sy...,406.0,'OS: Windows 7',67,'Memory: 2 GB RAM',191.0,'Memory: 4 GB RAM',165.0,'DirectX: Version 9.0c',136.0,'DirectX: Version 11',145.0,'Network: Broadband Internet connection',124.0,' Requires a 64-bit processor and operating s...,103.0,' Requires a 64-bit processor and operating s...,108.0,' Requires a 64-bit processor and operating s...,97.0,'Memory: 4 GB RAM',80.0,' Requires a 64-bit processor and operating s...,70.0,'DirectX: Version 11',93.0,'DirectX: Version 11',92.0,'DirectX: Version 11',76.0,'DirectX: Version 11',77.0,'Network: Broadband Internet connection',82.0,'Network: Broadband Internet connection',66.0,'Network: Broadband Internet connec

## metacritic_products

In [25]:
metacritic_products = pd.read_csv("../data/crawled_data/metacritic_products.csv")

In [26]:
metacritic_products.dtypes

appname           object
num_of_players    object
rating            object
dtype: object

In [27]:
# Do not need number of players, moreover, lots of games were not rated, or just null. Therefore, I'll consider to use this dataset
metacritic_products

,appname,num_of_players,rating
0,Counter-Strike: Global Offensive,NaN,M
1,Dota 2,Up to 10,T
2,PlayerUnknown's Battlegrounds,Up to more than 64,M
3,Grand Theft Auto V,Up to 32,M
4,Apex Legends,Online Multiplayer,T
...,...,...,...
3785,The Last Error,NaN,NaN
3786,Escape From Nazi Labs,NaN,NaN
3787,Project First Contact,NaN,NaN
3788,All Alone: VR,NaN,NaN


Big publisher infos are collected from metacritic

In [28]:
big_pubs = pd.read_csv("../data/additional_data/bigpublishers.csv")

In [29]:
big_pubs

,bigpublishers
0,505 Games
1,Activision
2,Paradox Interactive
3,Capcom
4,Annapurna Interactive
5,Xbox Game Studios
6,Microsoft Studios
7,Humble Games
8,Square Enix
9,Devolver Digital


## aggregate appinfo dataset

In [30]:
# add big publisher data
pubs = list(big_pubs["bigpublishers"])
appinfo["bigpublishers"] = appinfo["publisher"].isin(pubs).astype(int)

In [31]:
gameinfo = pd.merge(applist,appinfo, how="inner", on="appid")
gameinfo

,appid,appname,alltimepick,current,title,release_date,genres,developer,publisher,specs,user_tags,sys_requirement,Action,Indie,Strategy,Early Access,Adventure,Casual,Indie,Simulation,RPG,Free to Play,Racing,Single-player,Online PvP,MMO,Cross-Platform Multiplayer,Online Co-op,Multiplayer,Co-op,playwithother,bigpublishers
0,730,Counter-Strike: Global Offensive,1308963,350403,Counter-Strike: Global Offensive,2012-08-21,"[Action, Free to Play]",Valve,Valve,"[Steam Achievements, Full controller support, ...","[FPS, Shooter, Multiplayer, Competitive, Actio...","[['OS: Windows® 7/Vista/XP', 'Processor: Inte...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1
1,570,Dota 2,1295114,301215,Dota 2,2013-07-09,"[Action, Free to Play, Strategy]",Valve,Valve,"[Steam Trading Cards, Steam Workshop, SteamVR ...","[Free to Play, MOBA, Multiplayer, Strategy, eS...","[['OS: Windows 7 or newer', 'Processor: Dual ...",1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,3257248,150919,PUBG: BATTLEGROUNDS,2017-12-21,"[Action, Adventure, Massively Multiplayer]","KRAFTON, Inc.","KRAFTON, Inc.","[Online PvP, Stats, Remote Play on Phone, Remo...","[Survival, Shooter, Multiplayer, Battle Royale...",[['Requires a 64-bit processor and operating s...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0
3,1172470,Apex Legends,330879,113450,Apex Legends™,2020-11-04,"[Action, Adventure, Free to Play]",Respawn Entertainment,Electronic Arts,"[Online PvP, Online Co-op, Steam Achievements,...","[Free to Play, Battle Royale, Multiplayer, Sho...",[['A Roster of Legendary Characters -Master a ...,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0,1,1
4,440,Team Fortress 2,151253,71455,Team Fortress 2,2007-10-10,"[Action, Free to Play]",Valve,Valve,"[Cross-Platform Multiplayer, Steam Achievement...","[Free to Play, Hero Shooter, Multiplayer, FPS,...","[['OS: Windows® 7 (32/64-bit)/Vista/XP', 'Pro...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,604830,All Alone: VR,1005,0,All Alone: VR,2017-03-08,"[Adventure, Indie]",8bit,Bitlock Studio,"[Single-player, Steam Trading Cards]","[Adventure, Indie, VR]","[['OS: Win 7', 'Processor: i5', 'Memory: 4 G...",0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
3514,271550,Tribloos 2,1004,0,Tribloos 2,2013-01-12,"[Casual, Indie, Strategy]",BumpkinBrothers,BumpkinBrothers,"[Single-player, Steam Achievements, Steam Trad...","[Casual, Indie, Strategy, Time Management]",[['Beautiful Hand Drawn graphicsOver 90 levels...,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0
3515,336520,Might & Magic Heroes Online,1003,0,Might & Magic Heroes Online,2015-11-24,"[Free to Play, Massively Multiplayer, RPG, Str...",Blue Byte,Ubisoft,"[Single-player, MMO, Steam Trading Cards]","[Free to Play, Strategy, Massively Multiplayer...","[['OS: Windows XP SP3', 'Processor: 2 GHz Dua...",0,0,1,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1
3516,437390,HoPiKo,1001,0,HoPiKo,2017-01-06,"[Action, Casual, Indie]",Laser Dog,Merge Games,"[Single-player, Steam Achievements, Full contr...","[Indie, Casual, Action, Platformer, Great Soun...",[['Fast and brutal twitch-based gameplay.Aweso...,1,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0


# Preprocess panel dataset

- Resample historical data on daily level, weekly level or even monthly level
    - Time data type to datetime
    - appdemand.pickle : basic preprocess, calculate marketshare
    - price_sale.pickle : fill ommited time period
    - update_history.pickle : basic preprocess
    - metacritic_user_reviews : language detect & translate to english



In [32]:
appdemand = pd.read_pickle("../data/crawled_data/appdemand.pickle")

#### appdemand

In [33]:
appdemand.dtypes

appid                    object
time             datetime64[ns]
value                   float64
values_twitch           float64
dtype: object

In [34]:
appdemand

,appid,time,value,values_twitch
0,730,2011-11-30,680.0,NaN
1,730,2011-12-01,NaN,NaN
2,730,2011-12-02,NaN,NaN
3,730,2011-12-03,NaN,NaN
4,730,2011-12-04,NaN,NaN
...,...,...,...,...
1453,686230,2021-07-23,NaN,NaN
1454,686230,2021-07-24,NaN,NaN
1455,686230,2021-07-25,NaN,NaN
1456,686230,2021-07-26,NaN,NaN


calculate marketshare
- individual value/total value in each time period

In [42]:
# calc daily market size
total = appdemand.groupby("time").sum().reset_index()[["time","value","values_twitch"]]
total = total.rename(columns={'value':"total_value",'values_twitch':"total_values_twitch"})

# merge market size
appdemand = pd.merge(appdemand,total, how="left", on="time")

# calc market_share
appdemand["market_share"] = appdemand["value"]/total["total_value"]
appdemand["market_share_twitch"] = appdemand["values_twitch"]/total["total_values_twitch"]
appdemand

,appid,time,value,values_twitch,total_value_x,total_values_twitch_x,total_value_y,total_values_twitch_y,total_value_x,total_values_twitch_x,market_share,market_share_twitch,total_value_y,total_values_twitch_y,total_value,total_values_twitch
0,730,2011-11-30,680.0,NaN,624690.0,0.0,624690.0,0.0,624690.0,0.0,0.027017,NaN,624690.0,0.0,624690.0,0.0
1,730,2011-12-01,NaN,NaN,612419.0,0.0,612419.0,0.0,612419.0,0.0,NaN,NaN,612419.0,0.0,612419.0,0.0
2,730,2011-12-02,NaN,NaN,733713.0,0.0,733713.0,0.0,733713.0,0.0,NaN,NaN,733713.0,0.0,733713.0,0.0
3,730,2011-12-03,NaN,NaN,792417.0,0.0,792417.0,0.0,792417.0,0.0,NaN,NaN,792417.0,0.0,792417.0,0.0
4,730,2011-12-04,NaN,NaN,757220.0,0.0,757220.0,0.0,757220.0,0.0,NaN,NaN,757220.0,0.0,757220.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9361505,686230,2021-07-23,NaN,NaN,6373771.0,2977072.0,6373771.0,2977072.0,6373771.0,2977072.0,NaN,NaN,6373771.0,2977072.0,6373771.0,2977072.0
9361506,686230,2021-07-24,NaN,NaN,6680464.0,2527338.0,6680464.0,2527338.0,6680464.0,2527338.0,NaN,NaN,6680464.0,2527338.0,6680464.0,2527338.0
9361507,686230,2021-07-25,NaN,NaN,6889312.0,2701398.0,6889312.0,2701398.0,6889312.0,2701398.0,NaN,NaN,6889312.0,2701398.0,6889312.0,2701398.0
9361508,686230,2021-07-26,NaN,NaN,6366395.0,2778615.0,6366395.0,2778615.0,6366395.0,2778615.0,NaN,NaN,6366395.0,2778615.0,6366395.0,2778615.0


#### Resample appdemand 

In [43]:
# resample to week
appdemand_week = appdemand.groupby(by="appid").resample("W",on="time").mean().reset_index()

total_week= appdemand_week.groupby("time").sum().reset_index()[["time",
                                               "value",
                                               "values_twitch"]].rename(columns={'value':"total_value",
                                                                                 'values_twitch':"total_values_twitch" })
appdemand_week = pd.merge(appdemand_week,total_week, how="left", on="time")

appdemand_week["market_share"] = appdemand_week["value"]/total_week["total_value"]
appdemand_week["market_share_twitch"] = appdemand_week["values_twitch"]/total_week["total_values_twitch"]
appdemand_week

,appid,time,value,values_twitch,total_value_x,total_values_twitch_x,total_value_y,total_values_twitch_y,total_value_x,total_values_twitch_x,market_share,market_share_twitch,total_value_y,total_values_twitch_y,total_value_x,total_values_twitch_x,total_value_y,total_values_twitch_y
0,10,2004-01-18,23241.000000,NaN,5.033800e+03,0.000000e+00,5.033800e+03,0.000000e+00,5.033800e+03,0.000000e+00,0.923398,NaN,5.033800e+03,0.000000e+00,5.033800e+03,0.000000e+00,2.516900e+04,0.000000e+00
1,10,2004-01-25,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,10,2004-02-01,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,10,2004-02-08,39709.000000,NaN,6.494857e+03,0.000000e+00,6.494857e+03,0.000000e+00,6.494857e+03,0.000000e+00,0.873416,NaN,6.494857e+03,0.000000e+00,6.494857e+03,0.000000e+00,4.546400e+04,0.000000e+00
4,10,2004-02-15,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342763,999730,2021-07-04,1.571429,NaN,6.820233e+06,2.879689e+06,6.820233e+06,2.879689e+06,6.820233e+06,2.879689e+06,NaN,NaN,6.820233e+06,2.879689e+06,6.820233e+06,2.879689e+06,6.822674e+06,2.879692e+06
1342764,999730,2021-07-11,1.428571,NaN,6.679113e+06,3.206298e+06,6.679113e+06,3.206298e+06,6.679113e+06,3.206298e+06,NaN,NaN,6.679113e+06,3.206298e+06,6.679113e+06,3.206298e+06,6.707594e+06,3.206303e+06
1342765,999730,2021-07-18,1.571429,NaN,6.403773e+06,2.990567e+06,6.403773e+06,2.990567e+06,6.403773e+06,2.990567e+06,NaN,NaN,6.403773e+06,2.990567e+06,6.403773e+06,2.990567e+06,6.420450e+06,2.990782e+06
1342766,999730,2021-07-25,1.285714,NaN,6.406843e+06,2.735972e+06,6.406843e+06,2.735972e+06,6.406843e+06,2.735972e+06,NaN,NaN,6.406843e+06,2.735972e+06,6.406843e+06,2.735972e+06,6.417546e+06,2.736016e+06


#### price_sale

In [44]:
price_sale = pd.read_pickle("../data/crawled_data/price_sale.pickle")

In [45]:
price_sale.dtypes

time              object
price            float64
price_us          object
discount_rate    float64
appid             object
discount_name     object
dtype: object

In [46]:
# to datetime
price_sale["time"] = pd.to_datetime(price_sale["time"], infer_datetime_format=True)

In [47]:
# set appid as index
df = price_sale.set_index("appid")
df

,time,price,price_us,discount_rate,discount_name
appid,,,,,
730,2012-11-02,9.99,$ 9.99,33.0,NaN
730,2012-11-22,11.24,$ 11.24,25.0,NaN
730,2012-11-27,7.49,$ 7.49,50.0,NaN
730,2012-12-21,11.24,$ 11.24,25.0,Winter Sale 2012
730,2012-12-28,7.49,$ 7.49,50.0,NaN
...,...,...,...,...,...
686230,2019-10-29,0.49,$ 0.49,51.0,Halloween Sale 2019
686230,2019-11-02,0.99,$ 0.99,0.0,NaN
686230,2019-11-19,0.49,$ 0.49,51.0,NaN


In [48]:
# select random game for test case
idx730 = df[df.index == '730']
idx730.head()

,time,price,price_us,discount_rate,discount_name
appid,,,,,
730,2012-11-02,9.99,$ 9.99,33.0,NaN
730,2012-11-22,11.24,$ 11.24,25.0,NaN
730,2012-11-27,7.49,$ 7.49,50.0,NaN
730,2012-12-21,11.24,$ 11.24,25.0,Winter Sale 2012
730,2012-12-28,7.49,$ 7.49,50.0,NaN


In [49]:
reidx730 = df[df.index == '730'].reset_index()
reidx730.head()

,appid,time,price,price_us,discount_rate,discount_name
0,730,2012-11-02,9.99,$ 9.99,33.0,NaN
1,730,2012-11-22,11.24,$ 11.24,25.0,NaN
2,730,2012-11-27,7.49,$ 7.49,50.0,NaN
3,730,2012-12-21,11.24,$ 11.24,25.0,Winter Sale 2012
4,730,2012-12-28,7.49,$ 7.49,50.0,NaN


In [50]:
# start time
start = df[df.index == '730']["time"][0]
# end time
end = df[df.index == '730']["time"][-1]
# make time period
days = pd.DataFrame(pd.date_range(start, end, freq="D")).rename(columns={0:"time"})
days

,time
0,2012-11-02
1,2012-11-03
2,2012-11-04
3,2012-11-05
4,2012-11-06
...,...
2222,2018-12-03
2223,2018-12-04
2224,2018-12-05
2225,2018-12-06


In [51]:
# merge time period to fill the ommited time
pd.merge(days,reidx730, how="left",on="time").fillna(method="ffill")

,time,appid,price,price_us,discount_rate,discount_name
0,2012-11-02,730,9.99,$ 9.99,33.0,NaN
1,2012-11-03,730,9.99,$ 9.99,33.0,NaN
2,2012-11-04,730,9.99,$ 9.99,33.0,NaN
3,2012-11-05,730,9.99,$ 9.99,33.0,NaN
4,2012-11-06,730,9.99,$ 9.99,33.0,NaN
...,...,...,...,...,...,...
2232,2018-12-03,730,14.99,$ 14.99,0.0,Autumn Sale 2018 (Steam Awards Nominations)
2233,2018-12-04,730,14.99,$ 14.99,0.0,Autumn Sale 2018 (Steam Awards Nominations)
2234,2018-12-05,730,14.99,$ 14.99,0.0,Autumn Sale 2018 (Steam Awards Nominations)
2235,2018-12-06,730,14.99,$ 14.99,0.0,Autumn Sale 2018 (Steam Awards Nominations)


In [53]:
# load gameinfo dataset preprocessed from appinfo
game_info = pd.read_pickle("../data/preprocessed_data/game_info.pickle")

In [54]:
# currently free game list: 460 among 3518
current_free = game_info[game_info["Free to Play"] == 1]["appid"].unique()

In [55]:
# for each appid, filter price history record more than 1 and price is 0 on last date
turn_free_df = pd.DataFrame() # games that turn free and keep it free

for appid in price_sale["appid"].unique():
    price_history = price_sale[price_sale["appid"] == appid].shape[0]
    last_day = price_sale[price_sale["appid"] == appid][-1:]
    last_price = float(last_day["price"][-1:])

    if (price_history > 1) & (last_price == 0):
        turn_free_df = pd.concat([turn_free_df,last_day])
turn_free_df.head()

,time,price,price_us,discount_rate,appid,discount_name
86,2018-12-07,0.0,$ 0.00,0.0,730,NaN
5,2014-11-05,0.0,$ 0.00,0.0,570,NaN
149,2021-01-21,0.0,$ 0.00,0.0,271590,NaN
3,2019-11-22,0.0,$ 0.00,0.0,1085660,NaN
87,2020-09-24,0.0,$ 0.00,0.0,252950,NaN


In [56]:
# turn free game lists
turn_free = turn_free_df["appid"].unique() 

In [57]:
# originally freegame = difference of sets of currently freegame, turn free game.
originally_free= np.setdiff1d(current_free,turn_free)

/Users/wonjae/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:579: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


In [58]:
# price_sale history df excluding originally free games
price_sale_adj_df= price_sale[~price_sale["appid"].isin(originally_free)]

In [59]:
original_price_adj_df = price_sale_adj_df[price_sale_adj_df["discount_rate"] == 0].drop_duplicates(subset = ["price","appid"])

In [60]:
free_game_df = pd.DataFrame()

for appid in original_price_adj_df["appid"].unique():
    history = original_price_adj_df[original_price_adj_df["appid"] == appid].shape[0]

    if history == 1:
        price_check = float(original_price_adj_df[original_price_adj_df["appid"] == appid]["price"])

        if price_check == 0.0:
            free_game_df = pd.concat([free_game_df,original_price_adj_df[original_price_adj_df["appid"] == appid]])      

In [61]:
orignally_free_games_ls = list(originally_free)
orignally_free_games_ls.extend(list(free_game_df["appid"]))

In [62]:
# only contain paid game's price sale history (exclude turn free, originally free games)
price_sale_adj2_df= price_sale[~price_sale["appid"].isin(orignally_free_games_ls)]

In [63]:
# get original price
original_price_adj2_df = price_sale_adj2_df[price_sale_adj2_df["discount_rate"] == 0].drop_duplicates(subset = ["price","appid"])

In [64]:
# get released price
release_price_df = original_price_adj2_df.drop_duplicates(subset = ["appid"])
# Dataset without orginally freegame (price, discount records in each turning point)
price_sale_adj2_df = pd.merge(price_sale_adj2_df,release_price_df[["appid","price"]], how= "left", on="appid").rename(columns={"price_x":"price",
                                                                                                    "price_y":"release_price"})
# discount rate based on released price
price_sale_adj2_df["release_discount_rate"]= round(1 - (price_sale_adj2_df["price"]/price_sale_adj2_df["release_price"]),2) * 100

### add additional info on game_info

In [66]:
turn_free_df.dtypes

time             datetime64[ns]
price                   float64
price_us                 object
discount_rate           float64
appid                    object
discount_name            object
dtype: object

In [69]:
turn_free_df["appid"] = turn_free_df["appid"].astype(int)

In [70]:
# update gameinfo
game_info["originally_free_game"] = np.where(game_info["appid"].isin(orignally_free_games_ls),1,0)
game_info["turn_free_game"] = np.where(game_info["appid"].isin(turn_free),1,0)
game_info = pd.merge(game_info, turn_free_df[["appid","time"]], how="left",on="appid").rename(columns={"time":"turn_free_time"})

#### update history

In [71]:
update_history = pd.read_pickle("../data/crawled_data/update_history.pickle")

In [72]:
update_history.dtypes

DATE            object
DAY             object
TIME            object
PATCH TITLE     object
BUILDID        float64
appid            int64
dtype: object

In [73]:
# to datetime
update_history["DATE"] = pd.to_datetime(update_history["DATE"], infer_datetime_format=True)
update_history

,DATE,DAY,TIME,PATCH TITLE,BUILDID,appid
0,2021-07-27,Tue,23:14,1.37.9.7 (version 1306),7097051.0,730
1,2021-07-22,Thu,00:31,1.37.9.6 (version 1305),7064453.0,730
2,2021-07-07,Wed,21:55,1.37.9.5 (version 1304),6995305.0,730
3,2021-07-06,Tue,23:29,1.37.9.5 (version 1301),6989269.0,730
4,2021-06-24,Thu,00:54,1.37.9.4 (version 1292),6923374.0,730
...,...,...,...,...,...,...
52831,2016-11-08,Tue,15:58,No title,1444844.0,524680
52832,2017-08-25,Fri,15:54,No title,2068611.0,634230
52833,2017-07-27,Thu,14:34,No title,1991619.0,634230
52834,2017-07-26,Wed,07:13,No title,1985122.0,634230


In [74]:
update_history = update_history.drop_duplicates(["appid","DATE"])
update_history["update"] = 1
update_history["update_cum"]= update_history[["appid","DATE","update"]].groupby(["appid"]).cumcount(ascending=False) + 1

<ipython-input-74-d8245fa6baf0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  update_history["update"] = 1
<ipython-input-74-d8245fa6baf0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  update_history["update_cum"]= update_history[["appid","DATE","update"]].groupby(["appid"]).cumcount(ascending=False) + 1


In [75]:
# resampe to week
update_history_W = update_history.groupby("appid").resample("W",on="DATE").count()["update"].reset_index()
update_history_W["update_cum"]= update_history_W.groupby(["appid"])["update"].cumsum()

---

#### metacritic_user_reviews

- Tag appid (join appid from applist)
1. Resample review rating score in week, monthly 
2. write code for langauge detext and translate incase for text mining 
3. user info preprocess if needed later

In [86]:
applist["metacritic_name"]= applist["appname"].apply(make_url_name)
applist

,appid,appname,alltimepick,current,metacritic_name
0,730,Counter-Strike: Global Offensive,1308963,350403,counter-strike-global-offensive
1,570,Dota 2,1295114,301215,dota-2
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,3257248,150919,playerunknowns-battlegrounds
3,271590,Grand Theft Auto V,364548,124679,grand-theft-auto-v
4,1172470,Apex Legends,330879,113450,apex-legends
...,...,...,...,...,...
4814,619730,Mad Driver,1002,0,mad-driver
4815,437390,HoPiKo,1001,0,hopiko
4816,524680,Escape From Nazi Labs,1000,0,escape-from-nazi-labs
4817,634230,Project First Contact,1000,0,project-first-contact


In [6]:
metacritic_user_reviews = pd.read_pickle("../data/crawled_data/metacritic_user_reviews.pickle")

In [61]:
metacritic_user_reviews.dtypes

metacritic_name    object
user_name          object
date               object
rating             object
review             object
dtype: object

In [89]:
metacritic_user_reviews[metacritic_user_reviews.notnull()].count()

metacritic_name    188772
user_name          170732
date               185661
rating             185661
review             185641
dtype: int64

In [90]:
metacritic_user_reviews.dropna(inplace=True)

In [91]:
metacritic_user_reviews[metacritic_user_reviews.notnull()].count()

metacritic_name    170712
user_name          170712
date               170712
rating             170712
review             170712
dtype: int64

In [92]:
metacritic_user_reviews["rating"] = metacritic_user_reviews["rating"].astype(int)
metacritic_user_reviews["date"] = pd.to_datetime(metacritic_user_reviews["date"], infer_datetime_format=True)
metacritic_user_reviews

,metacritic_name,user_name,date,rating,review
0,counter-strike-global-offensive,defenz,2021-08-08,7,⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣...
1,counter-strike-global-offensive,nomeoriginale,2021-08-06,4,"много читеров , бустеров , злых школьников , с..."
2,counter-strike-global-offensive,thepigeonman3,2021-08-05,0,"absolute **** game, **** boring maps, **** sho..."
3,counter-strike-global-offensive,Sherbakoff,2021-07-29,8,"Nice game, but there is a lot of cheaters, tox..."
4,counter-strike-global-offensive,Daun228,2021-07-25,0,Это старое графическое обновления для ещё стар...
...,...,...,...,...,...
185656,hex-shards-of-fate,HappyPenguin,2020-07-16,9,"Very simple, funny, cute and colourful ""point ..."
185657,hex-shards-of-fate,Fiend_Zero,2019-10-29,5,"I really wanted to like ***, but it just didn'..."
185658,hex-shards-of-fate,robinson427,2019-08-22,9,"This game was fun. The writing was funny, the ..."
185659,hex-shards-of-fate,rogueliker,2016-11-20,10,"Nice game , best underrated roguelike of the p..."


In [93]:
metacritic_user_reviews.dtypes

metacritic_name            object
user_name                  object
date               datetime64[ns]
rating                      int64
review                     object
dtype: object

In [94]:
# Tag appid (join appid from applist)
metacritic_user_reviews = pd.merge(metacritic_user_reviews,applist[["metacritic_name",'appid']], how='inner', on="metacritic_name")

,metacritic_name,user_name,date,rating,review,appid
0,counter-strike-global-offensive,defenz,2021-08-08,7,⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣⁣...,730
1,counter-strike-global-offensive,nomeoriginale,2021-08-06,4,"много читеров , бустеров , злых школьников , с...",730
2,counter-strike-global-offensive,thepigeonman3,2021-08-05,0,"absolute **** game, **** boring maps, **** sho...",730
3,counter-strike-global-offensive,Sherbakoff,2021-07-29,8,"Nice game, but there is a lot of cheaters, tox...",730
4,counter-strike-global-offensive,Daun228,2021-07-25,0,Это старое графическое обновления для ещё стар...,730
...,...,...,...,...,...,...
172164,hex-shards-of-fate,HappyPenguin,2020-07-16,9,"Very simple, funny, cute and colourful ""point ...",410380
172165,hex-shards-of-fate,Fiend_Zero,2019-10-29,5,"I really wanted to like ***, but it just didn'...",410380
172166,hex-shards-of-fate,robinson427,2019-08-22,9,"This game was fun. The writing was funny, the ...",410380
172167,hex-shards-of-fate,rogueliker,2016-11-20,10,"Nice game , best underrated roguelike of the p...",410380


In [34]:
# resample time period(week level)
week = metacritic_user_reviews.groupby("metacritic_name").resample("W",on="date").mean().reset_index()
week

,metacritic_name,date,rating
0,$1-ride,2012-12-09,1.000000
1,$1-ride,2012-12-16,NaN
2,$1-ride,2012-12-23,NaN
3,$1-ride,2012-12-30,NaN
4,$1-ride,2013-01-06,0.000000
...,...,...,...
290726,魔物讨伐团,2021-06-20,NaN
290727,魔物讨伐团,2021-06-27,NaN
290728,魔物讨伐团,2021-07-04,NaN
290729,魔物讨伐团,2021-07-11,3.000000


In [36]:
week["cum_mean"]= panel_cum_mean(week)
week

<ipython-input-33-b70f6f843734>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy["cumcount"] = copy.groupby(by="metacritic_name").cumcount()
<ipython-input-33-b70f6f843734>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy["cumsum"] = copy.groupby(by="metacritic_name")["rating"].cumsum()
<ipython-input-33-b70f6f843734>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,metacritic_name,date,rating,cum_mean
0,$1-ride,2012-12-09,1.000000,1.000000
1,$1-ride,2012-12-16,NaN,1.000000
2,$1-ride,2012-12-23,NaN,1.000000
3,$1-ride,2012-12-30,NaN,1.000000
4,$1-ride,2013-01-06,0.000000,0.500000
...,...,...,...,...
290726,魔物讨伐团,2021-06-20,NaN,7.083333
290727,魔物讨伐团,2021-06-27,NaN,7.083333
290728,魔物讨伐团,2021-07-04,NaN,7.083333
290729,魔物讨伐团,2021-07-11,3.000000,6.629630


In [37]:
panel_cum_mean(metacritic_user_reviews)

0           7.000000
1           6.565749
2           7.000000
3           6.565749
4           5.500000
              ...   
19842901    6.750000
19842902    6.647059
19842903    6.777778
19842904    6.947368
19842905    6.650000
Name: rating_cummean, Length: 19842906, dtype: float64

In [12]:
# for language detect and translate
import googletrans
from googletrans import Translator
from fake_useragent import UserAgent

In [14]:
# test for langauge detect and translate
text = "hi"
google_trans_urls = ["translate.google.com", "translate.google.co.kr"]
translator = Translator(service_urls=google_trans_urls, user_agent=UserAgent().chrome)
translations = translator.translate(text)
lang = translator.detect(text)
print(lang.lang)
print(translations.text)

en
hi


In [146]:
# detect langauge and translate
def detect_toEN(text, option):
    """
    Language detection & Translation using googletrans.
    text: 'str'
    option: 'str' detect or toEN
    """
    google_trans_urls = ["translate.google.com", "translate.google.co.kr"]
    translator = Translator(service_urls=google_trans_urls, user_agent=UserAgent().chrome)
    if option == "detect":
        try:
            lang = translator.detect(text)
            return lang.lang
        except:
            return np.nan
            
    elif option == "toEN":
        try:
            lang = translator.detect(text)
            if lang.lang != "EN":
                translations = translator.translate(text)
                return translations.text
            else:
                return text
            
        except:
            return np.nan

In [31]:
metacritic_user_reviews_preprocess("W",save=False)

/Users/wonjae/Google Drive/steam_project/data_alaysis/preprocess.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy["cumcount"] = copy.groupby(by="metacritic_name").cumcount()
/Users/wonjae/Google Drive/steam_project/data_alaysis/preprocess.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy["cumsum"] = copy.groupby(by="metacritic_name")["rating"].cumsum()
/Users/wonjae/Google Drive/steam_project/data_alaysis/preprocess.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,metacritic_name,date,rating,cum_mean
0,$1-ride,2012-12-09,1.000000,1.000000
1,$1-ride,2012-12-16,NaN,1.000000
2,$1-ride,2012-12-23,NaN,1.000000
3,$1-ride,2012-12-30,NaN,1.000000
4,$1-ride,2013-01-06,0.000000,0.500000
...,...,...,...,...
290726,魔物讨伐团,2021-06-20,NaN,7.083333
290727,魔物讨伐团,2021-06-27,NaN,7.083333
290728,魔物讨伐团,2021-07-04,NaN,7.083333
290729,魔物讨伐团,2021-07-11,3.000000,6.629630
